根據把首頁往下滑，發現網站會對server送一個GET請求下一個30篇文章
- 2018(G) https://ithelp.ithome.com.tw/articles/10203216
- 2019(G) https://tlyu0419.github.io/2019/04/06/Crawl-Dcard/
- 2020 https://blog.jiatool.com/posts/dcard_api_v2/

### API root

- 看板資訊 http://dcard.tw/_api/forums
- 文章資訊 http://dcard.tw/_api/forums/{看板名稱}/posts
- 文章內文	http://dcard.tw/_api/posts/{文章編號}
- 文章內引用連結	http://dcard.tw/_api/posts/{文章編號}/links
- 文章內留言	http://dcard.tw/_api/posts/{文章編號}/comments

### API wirte
- 文章數量 https://www.dcard.tw/_api/posts?limit=100
- 文章分頁 http://dcard.tw/_api/posts/?before=文章id (before 回傳由新到舊 after舊到新)
- 文章留言 http://dcard.tw/_api/posts/文章id/comments
- 留言分頁 http://dcard.tw/_api/posts/文章id/comments?after=floor數字
- 熱門文章 https://dcard.tw/_api/posts?popular=true

### others
文章資訊(meta) 與 文章內留言 預設使用熱門度 (popularity) 作為排序，而且一次請求 (request) 中只回應 30 筆。
* 想要讓這兩項使用 時間 排序，可在 GET 參數中加入 popular=false
* 欲取得更多的 文章資訊(meta)，可以使用 before={某文章編號}來獲得早於 #{某文章編號} 的另外 30 筆 資訊。
* 欲取得更多的 文章內留言，可以使用 after={某樓層} 來獲得大於 #{某樓層} 的另外 30 筆 留言。

In [1]:
import requests
import json
import pandas as pd    # 引用套件並縮寫為 pd
from bs4 import BeautifulSoup
# https://www.dcard.tw/f/makeup

GET 美妝版首頁，Use BeautifulSoup with html, wrtie a txt

 

In [2]:
p = requests.Session()                             # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/f/makeup")  # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
#sel

#for link in sel:
#    a.append(link["href"])
    #print('https://www.dcard.tw'+link.get('href'))

In [3]:
# slope the title
def Catch(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [4]:
# KeyError: 'id'
# Oh！文章不見了 這篇文章已被作者刪除。 按這裡 看更多文章吧
# https://www.dcard.tw/f/makeup/p/235079399
# Catch(235079399)

In [5]:
# # 1w : 234227720
# # 2w : 233254532
# # 3w : 232166760
# # 4w : 231148015
# # 5w : 230149776
# url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=233254532" 

In [6]:
import random
import time

#url = "http://dcard.tw/_api/forums/makeup/posts?popular=false&limit=1"
url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=231148015" 
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(Catch(rejs[0]['id']),ignore_index=True) 
#df
print(str(int(df['id'].tail(1))) )

231148004


In [8]:
count = 0
delay_choices = [3, 5, 7]  # delay second
delay = random.choice(delay_choices) 

for j in range(1):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    print(last)
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        count = count+1
        print('data reading...',count,'id:',rejs[i]['id'],rejs[i]['title'])
    delay = random.choice(delay_choices)
    time.sleep(delay)  
df.to_csv('data/Dcard_makeup.csv', index=False,encoding = "UTF-8")
print(df.shape)
df

231137891
data reading... 1 id: 231137769 #分享 #阿皂屋體驗包
data reading... 2 id: 231137592 ＃圖 GA官網買黑絲絨粉餅霜＋唇膏快速到貨分享
data reading... 3 id: 231137536 李聖經代言隱眼CC（愛爾康）
data reading... 4 id: 231137535 Clart'e spa 珂萊緹
data reading... 5 id: 231137520 首次購入倩碧📌保濕平衡修護組
data reading... 6 id: 231137429 新手眉毛求批🙏🏻🙏🏻🙏🏻
data reading... 7 id: 231137282 有人知道這個品牌嗎
data reading... 8 id: 231137216 #文長 30天粉底挑戰💪 Part3
data reading... 9 id: 231137097 26樣長痘痘時會想買的保養品大補帖🌞🌿
data reading... 10 id: 231136854 Yes Please！日常眼妝分享
data reading... 11 id: 231136797 有沒有人買過solid state 香膏
data reading... 12 id: 231136794 #情報 買integrate彩妝組合抽Dyson造型器
data reading... 13 id: 231136777 🇰🇷亂亂買
data reading... 14 id: 231136643 關於洗臉時機
data reading... 15 id: 231136621 姊姊看到新東西 就買😂😂
data reading... 16 id: 231136574 台中霧眉#問
data reading... 17 id: 231136560 會因為唇彩的關係換深1號粉底嗎？
data reading... 18 id: 231136534 契爾氏Kiehl’s 唇部護理系列（護唇膏/美唇油） 💄不專業心得比較！
data reading... 19 id: 231136468 #粉紅控 美到冒泡的保養品們 (后/GA)
data reading... 20 id: 231136298 #有人買過shein的刷具嗎😂
dat

,id,title,content,excerpt,createdAt,updatedAt,forumName,forumAlias,gender,topics
0,231148004,#請益 無酒精的美白保養品,我的膚質是敏感肌，想找無酒精的美白保養品😳\n目前只知道BEVY C.美白是無酒精的\nht...,我的膚質是敏感肌，想找無酒精的美白保養品，目前只知道BEVY C.美白是無酒精的，上次看到有...,2019-04-23T06:54:01.936Z,2019-04-23T06:54:01.936Z,美妝,makeup,F,NaN
1,231147886,#閒聊 你心目中的刷具之王？,昨天收到一批受受狼 開箱用覺得刷毛好蘇福啊！\n但心目中刷具之王還是植村秀的55刷無誤！\n...,昨天收到一批受受狼 開箱用覺得刷毛好蘇福啊！，但心目中刷具之王還是植村秀的55刷無誤！，這把...,2019-04-23T06:34:34.241Z,2019-04-23T06:34:34.241Z,美妝,makeup,F,NaN
2,231147742,霧眉,想請問一下在基隆或台北有推薦霧眉的店家嗎？\n想去霧眉但找不到合適的店家\n希望有推薦的店家...,想請問一下在基隆或台北有推薦霧眉的店家嗎？想去霧眉但找不到合適的店家，希望有推薦的店家能與我...,2019-04-23T06:08:52.937Z,2019-04-23T06:08:52.937Z,美妝,makeup,F,NaN
3,231147734,#問 防曬需要卸嗎,我買了一瓶無色的防曬乳，上網查發現說要卸不卸的回答都有。\n想請問大家擦無色的純防曬乳會卸嗎...,我買了一瓶無色的防曬乳，上網查發現說要卸不卸的回答都有。想請問大家擦無色的純防曬乳會卸嗎？還...,2019-04-23T06:06:20.488Z,2019-04-23T06:06:20.488Z,美妝,makeup,F,NaN
4,231147726,雙眼皮膠水好有用！,一直以來我都大小眼一邊內雙一邊外雙，\n\nhttp://i.imgur.com/NNdZJ...,一直以來我都大小眼一邊內雙一邊外雙，某天照鏡子突然覺得欸 好像真的差滿多的，就隨便去寶雅買...,2019-04-23T06:05:07.541Z,2019-04-23T06:06:08.209Z,美妝,makeup,F,NaN
...,...,...,...,...,...,...,...,...,...,...
196,231129976,（實戴圖）cocoview自然灰色隱眼👀,大家晚安 第一次就在美妝版發文！！！\n\n原本一直都是戴透明片\n但是頭髮剪短之後風格變成...,大家晚安 第一次就在美妝版發文！！！，原本一直都是戴透明片，但是頭髮剪短之後風格變成可愛風，...,2019-04-20T13:49:29.591Z,2019-04-20T13:51:55.694Z,美妝,makeup,F,NaN
197,231129871,#innisfree 沒有最狂 只有更狂 送很大!!!!!🤣🤣🤣🤣🤣,—————更新可忽略 請下滑——————-\n歐抹歐抹，上熱門了\n真的嚇死寶寶了\n回覆裡...,—————更新可忽略 請下滑——————-，歐抹歐抹，上熱門了，真的嚇死寶寶了，回覆裡有水水...,2019-04-20T13:32:34.855Z,2019-04-21T14:44:37.841Z,美妝,makeup,F,NaN
198,231129669,#圖 ELLE路跑完賽禮分享 #優若美折扣碼 #初時肌,這週一直下雨\n原本以為週六ELLE的路跑要泡湯了\n還好天氣不錯☀️☀️☀️\n\n順利跑...,這週一直下雨，原本以為週六ELLE的路跑要泡湯了，還好天氣不錯️️️，順利跑完後，到攤位排隊...,2019-04-20T12:59:06.510Z,2019-04-20T12:59:06.510Z,美妝,makeup,F,NaN
199,231129651,發問:屈臣氏特定會員85折折扣怪怪的,如題\n我抽到了特定會員開架彩妝85折的簡訊\nhttps://i.imgur.com/WE...,如題，我抽到了特定會員開架彩妝85折的簡訊，然後今天就開心的跑去買了，問題來了Integra...,2019-04-20T12:55:41.777Z,2019-04-20T12:55:41.777Z,美妝,makeup,F,NaN


# ---------------正式版本分隔線---------------
- 測試版本代號：1.2  2020
- 正式版本代號：2.03  2021
- 開發人員： EGG Jerry Three


dva.x_x 版本開發

In [ ]:
#找到 dav.x_x 發表過的文章

p = requests.Session()                             # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/@dva.x_x")  # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
sel
a=[]
for link in sel:
    a.append(link["href"])
    print(link.get('href')[-9:])
    print('https://www.dcard.tw'+link.get('href'))

In [ ]:
# slope the title
def CatchUser(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'department':rejs['department'],  #for author
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'department', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'department':"",    #for author
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'department', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [ ]:
# 熱門美妝版測試 (100筆)
url = 'https://www.dcard.tw/@dva.x_x'
#url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=5'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(CatchUser(rejs[i]['id']),ignore_index=True)   #0 - 29
print(df.shape)
#df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df
# 99	235031315	#請益 #問 LA girl 遮瑕挑色
# 235066991

In [ ]:
import random
import time

url = "http://dcard.tw/_api/forums/makeup/posts?popular=false&limit=1"
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(CatchUser(rejs[0]['id']),ignore_index=True) 
#df
print(str(int(df['id'].tail(1))) )

count = 0
delay_choices = [1, 3, 5]  # delay second
delay = random.choice(delay_choices) 

for j in range(1):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    print(last)
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=false&limit=100&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        count = count+1
        print('data reading...',count,'id:',rejs[i]['id'],rejs[i]['title'])
    delay = random.choice(delay_choices)
    time.sleep(delay)  
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
print(df.shape)
df

In [ ]:
# slope the title
def CatchPost(num):
    try:
        url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
        resq = requests.get(url)
        rejs = resq.json()
        return(pd.DataFrame(
            data=
            [{'id':rejs['id'],
              'title':rejs['title'],
              'content':rejs['content'],
              'excerpt':rejs['excerpt'],
              'createdAt':rejs['createdAt'],
              'updatedAt': rejs['updatedAt' ],
              'forumName':rejs['forumName'],
              'forumAlias':rejs['forumAlias'],
              'gender':rejs['gender'],
              'topics':rejs['topics']}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )
    except:
        print('data error...',i,'J:',j)
        return(pd.DataFrame(
            data=
            [{'id':"",
              'title':"",
              'content':"",
              'excerpt':"",
              'createdAt':"",
              'updatedAt': "",
              'forumName':"",
              'forumAlias':"",
              'gender':"",
              'topics':""}],
            columns=['id', 'title', 'content', 'excerpt', 'createdAt', 
                     'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
              )

In [ ]:
#CatchPost(234999065)
#url = 'https://www.dcard.tw/_api/posts/'+str(num)  #api post with ID
url = 'https://www.dcard.tw/_api/posts/235103419'
resq = requests.get(url)
rejs = resq.json()
rejs

In [ ]:
url = 'https://www.dcard.tw/_api/posts/234912821'
resq = requests.get(url)
rejs = resq.json()
rejs

In [ ]:
# url = 'https://www.dcard.tw/_api/posts/235103302'
resq = requests.get(url)
rejs = resq.json()
rejs

# --------------- dva.x_x 版本分隔線 ---------------
- 測試版本代號：Jerry 1.0
- 開發人員： Jerry


In [ ]:
#test = open("data/test.txt","w",encoding='UTF-8')

In [ ]:
# asb
import random
import time

url = "http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=99"
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df = df.append(Catch(rejs[0]['id']),ignore_index=True)   #0 - 29

delay_choices = [1]#, 3, 5, 7,20]  #延遲的秒數
delay = random.choice(delay_choices)  #隨機選取秒數

for j in range(10):
    last = str(int(df['id'].tail(1))) # 找出爬出資料的最後一筆ID
    url = "https://www.dcard.tw/_api/forums/makeup/posts?popular=true&limit=99&before=" + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 101 == 0:
            print('第',j,'次執行，data reading...',i,'last',last,'chocice',delay)
    delay = random.choice(delay_choices)  #隨機選取秒數
    time.sleep(delay)  #延遲

df.to_csv('data/Op_Dcard_markeup.csv', index=False)
print(df.shape)
df

In [ ]:
# 熱門美妝版測試 (100筆)
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']),ignore_index=True)   #0 - 29
    if i % 20 == 0:
        print('data reading...',i)
print(df.shape)
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df
# 99	235031315	#請益 #問 LA girl 遮瑕挑色
# 235066991

In [ ]:
#last = str(int(df['ID'].tail(1))) #last dt
#last

In [ ]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(10):
    last = str(int(df['ID'].tail(1))) # 找出爬出資料的最後一筆ID
    url = 'https://www.dcard.tw/_api/posts?popular=true&before=' + last
    #url = 'https://www.dcard.tw/_api/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 10 == 0:
            print('第',j,'次執行，data reading...',i)
            
print(df.shape)
df

In [ ]:
last = str(int(df['ID'].tail(1)))
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
df

In [ ]:
# 熱門美妝版測試 (100筆)

for i in range(10):
    last = str(int(df['ID'].tail(1)))
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    df = pd.DataFrame()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']),ignore_index=True)
        if i % 20 == 0:
            print('data reading...',i)
    print(df.shape)

df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df

# 99	235031315	#請益 #問 LA girl 遮瑕挑色

In [ ]:
last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID '235031315'
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
resq = requests.get(url)
rejs = resq.json()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)


df

In [ ]:

#df

In [ ]:
##ok ^
for j in range(3):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)
#df

In [ ]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(10):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 20 == 0:
            print('data reading...',i)
print(df.shape)
df


In [ ]:
df.to_excel('data/Dcard_markeup.excel', index=False)

欄位	說明	備註
ID	編號	
title	標題	
content	內容	
excerpt	摘要	
createdAt	發布時間	
updatedAt	更新時間	
commentCount	留言數	
forumName	分類	中文
forumAlias	分類	英文
gender	性別	
likeCount	心情數量	
reactions	心情細項	把以上心情細分為「愛心」、「哈哈」、「跪」、「森77」、「驚訝」等類型
topics	標籤	

In [ ]:
# 固定碼測試
def Catch_test(ID):
    url = 'https://www.dcard.tw/_api/posts/'+str(ID)
    requ = requests.get(url)
    rejs = requ.json()
    #rejs
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'excerpt':rejs['excerpt'],
          'createdAt':rejs['createdAt'],
          'updatedAt': rejs['updatedAt' ],
          'forumName':rejs['forumName'],
          'forumAlias':rejs['forumAlias'],
          'gender':rejs['gender'],
          'topics':rejs['topics']}],
        columns=['ID', 'title', 'content', 'excerpt', 'createdAt', 
                 'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
          )
Catch_test(235020702)